In [9]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from early_stopping import EarlyStopping

import random
import numpy as np

random.seed(42)  # for consistent results

In [10]:
## Define training hyperparameters
lr = 0.001
momentum = 0.9
batch_size = 32
patience = 20
max_epochs = 100

In [11]:
## Load in training, validation and testing data
transform = transforms.Compose(
# convert PIL images in cifar10 dataset to tensors, normalize the images
# and augment them using random color modification and flips
[transforms.ToTensor(),
 transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
 transforms.ColorJitter(hue=.05, saturation=.05),
 transforms.RandomHorizontalFlip()]
)

# create dataloaders for train, valid and test data
# reserve 20% of training data for validation/early stopping
train_set = torchvision.datasets.CIFAR10(root='./data', train=True,
                                         download=True, transform=transform)

train_size = int(0.8 * len(train_set))
valid_size = len(train_set) - train_size

train_set, valid_set = torch.utils.data.random_split(train_set,
                                                     [train_size, valid_size])

test_set = torchvision.datasets.CIFAR10(root='./data', train=False,
                                        download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size,
                                           shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_set, batch_size=batch_size,
                                           shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size,
                                          shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [12]:
# Define LeNet (LeCun et al., 1998) network architecture
class LeNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=6,
                               kernel_size=5)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16,
                               kernel_size=5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        # first convolutional layer, w/ max pooling and ReLU activation
        x = self.pool(F.relu(self.conv1(x)))
        
        # second convolutional layer, same as first
        x = self.pool(F.relu(self.conv2(x)))
        
        # flatten output from second conv layer and pass thru
        # multi-layer perceptron for classification
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        
        # return logits from last layer
        return self.fc3(x)

In [14]:
## Define training loop
net = LeNet()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=lr, momentum=momentum)
early_stopping = EarlyStopping(patience=patience, verbose=True)

# keep track of validation set losses from each epoch
train_losses = []
valid_losses = []
avg_train_losses = []
avg_valid_losses = []

for epoch in range(1, max_epochs + 1):
    for i, data in enumerate(train_loader):
        inputs, labels = data
        optimizer.zero_grad()
        
        # get model predictions and calculate loss
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        
        # perform backprop using loss and model weights
        loss.backward()
        optimizer.step()
        
        # record training losses for this minibatch
        train_losses.append(loss.item())
        
    # after each training epoch, get average validation set loss and
    # do early stopping if necessary
    #
    # i.e: model has stopped improving on the validation set after
    # set patience
    with torch.no_grad():
        for inputs, labels in valid_loader:
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            valid_losses.append(loss.item())
    
    # calculate average training and average losses for this epoch
    train_loss = np.average(train_losses)
    valid_loss = np.average(valid_losses)
    avg_train_losses.append(train_loss)
    avg_valid_losses.append(valid_loss)
    
    print(f"Epoch {str(epoch)} | mean train loss: {train_loss:.5f} | mean valid loss: {valid_loss:.5f}\n")
    
    # reset list of train/valid losses for next epoch
    train_losses = []
    valid_losses = []
    
    # use EarlyStopping object to see if validation loss has stopped
    # improving/decreasing for the set patience
    early_stopping(valid_loss, net)
    
    if early_stopping.early_stop:
        print(f"Early stopping at epoch {str(epoch)} / {str(max_epochs)}")
        break

# load checkpoint with best model, created by EarlyStopping
net.load_state_dict(torch.load('checkpoint.pt'))

Epoch 1 | mean train loss: 2.28909 | mean valid loss: 2.23327

Validation loss decreased (inf --> 2.233272).  Saving model ...
Epoch 2 | mean train loss: 2.11701 | mean valid loss: 2.02885

Validation loss decreased (2.233272 --> 2.028853).  Saving model ...
Epoch 3 | mean train loss: 1.96398 | mean valid loss: 1.91477

Validation loss decreased (2.028853 --> 1.914765).  Saving model ...


KeyboardInterrupt: 

In [ ]:
## Test trained model
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = net(images)
        
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Network testing accuracy: {100 * correct // total} %')